In [187]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# Version 1

In [219]:
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')


def ver1():
    df = pd.DataFrame(
        np.hstack([inputs, targets]), 
        columns=['temp', 'rainfall', 'humidity', 'apples', 'oranges'],
    )
    display(df.head())
    x = torch.from_numpy(inputs) # (None, 3)
    y = torch.from_numpy(targets) # (None, 2)
    w = torch.randn(2, 3, requires_grad=True) # num_outputs, num_inputs
    b = torch.randn(2, requires_grad=True)

    def model(x):
        return x @ w.t() + b # (None, 2)

    def mse(y_pred, y_true):
        diff = y_pred - y_true
        return torch.sum(diff * diff) / diff.numel()

    for i in range(10000):
        y_pred = model(x)
        loss = mse(y, model(x)) 
        if i % 1000 == 0:
            print(i, loss.item())
        loss.backward()
        with torch.no_grad():
            w -= w.grad * 1e-5
            b -= b.grad * 1e-5
            w.grad.zero_()
            b.grad.zero_()
            
ver1()

,temp,rainfall,humidity,apples,oranges
0,73.0,67.0,43.0,56.0,70.0
1,91.0,88.0,64.0,81.0,101.0
2,87.0,134.0,58.0,119.0,133.0
3,102.0,43.0,37.0,22.0,37.0
4,69.0,96.0,70.0,103.0,119.0


0 23016.76953125
1000 13.044209480285645
2000 2.429203987121582
3000 0.7904254794120789
4000 0.5372937917709351
5000 0.498198002576828
6000 0.4921550452709198
7000 0.4912187159061432
8000 0.49107569456100464
9000 0.4910525381565094


# Version 2

In [221]:
def ver2():
    x = torch.from_numpy(inputs) # (None, 3)
    y = torch.from_numpy(targets) # (None, 2)
    train_ds = TensorDataset(x, y)
    # print(train_ds[0])
    train_dl = DataLoader(train_ds, batch_size=4, shuffle=True)
    for i in train_dl:
        print(i)
        break
        
    model = nn.Linear(in_features=3, out_features=2)
    # print(model.weight, model.bias)
    loss_fn = nn.functional.mse_loss
    # opt = torch.optim.SGD(model.parameters(), lr=1e-5)
    opt = torch.optim.Adam(model.parameters())
    print(loss_fn(model(x), y))
    
    def fit(num_epochs, model, loss_fn, opt):
        for epoch in range(num_epochs):
            for xb, yb in train_dl:
                pred = model(xb)
                loss = loss_fn(pred, yb)
                loss.backward() # compute gradients
                opt.step() # update parameters using gradients
                opt.zero_grad()
            if epoch % 1000 == 0:
                print(epoch, loss.item())
                
    fit(10000, model, loss_fn, opt)
                
ver2()

[tensor([[ 87., 134.,  58.],
        [ 73.,  67.,  43.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]]), tensor([[119., 133.],
        [ 56.,  70.],
        [ 22.,  37.],
        [103., 119.]])]
tensor(13066.3789, grad_fn=<MseLossBackward>)
0 11703.1015625
1000 123.7955322265625
2000 1.196951150894165
3000 0.5879496932029724
4000 0.41360726952552795
5000 0.62481689453125
6000 0.6183251142501831
7000 0.6007038950920105
8000 0.5651997327804565
9000 0.29849663376808167


In [196]:
model = nn.Linear(in_features=3, out_features=2)
print(model.weight, model.bias)
loss_fn = nn.functional.mse_loss

Parameter containing:
tensor([[ 0.3925, -0.4187, -0.3082],
        [ 0.5287, -0.1948, -0.2047]], requires_grad=True) Parameter containing:
tensor([-0.5586, -0.3306], requires_grad=True)


In [197]:
model(inputs)

AttributeError: 'numpy.ndarray' object has no attribute 'dim'